In [1]:
import os
import pandas as pd
import numpy as np
import json
import folium
from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import branca.colormap as cm
from matplotlib import colors

In [2]:
df_refugee = pd.read_csv("time_series.csv",low_memory=False)
display(df_refugee.size)
df_refugee['Value'] = pd.to_numeric(df_refugee['Value'], errors='coerce')
df_refugee= df_refugee[df_refugee['Value'] > 0]
df_refugee = df_refugee[df_refugee['Year']>1984]
df_refugee = df_refugee.groupby(['Year','Country / territory of asylum/residence','Origin']).sum().reset_index()

1492205

In [3]:
df_geo  = pd.read_csv("geo.csv")
df_merge = df_refugee.merge(df_geo, left_on='Country / territory of asylum/residence', right_on='name', how='inner')
df_merge = df_merge[['Year','Value','Origin',"country",'name']]
df_merge = df_merge.rename(columns={"country" : "country_asylum"})
df_merge = df_merge.merge(df_geo, left_on='Origin', right_on='name', how='inner')
df_merge = df_merge[['Year',"country","country_asylum",'Value','name_y','name_x']]
df_merge = df_merge.rename(columns={"country":"country_origin","name_y":"name_origin","name_x":"name_asylum"})
df_merge = df_merge.sort_values("Year")

display(df_merge.head())

,Year,country_origin,country_asylum,Value,name_origin,name_asylum
0,1985,CL,DZ,100.0,Chile,Algeria
86593,1985,ID,PG,11820.0,Indonesia,Papua New Guinea
62240,1985,LK,IT,10.0,Sri Lanka,Italy
32863,1985,GW,IT,2.0,Guinea-Bissau,Italy
53127,1985,ZW,BW,4250.0,Zimbabwe,Botswana


In [4]:
df_immigration_exit = df_merge.drop("country_asylum",1)
df_immigration_exit = df_immigration_exit.groupby(["Year","country_origin","name_origin"]).sum().reset_index()
df_immigration_exit= df_immigration_exit.sort_values(["country_origin","Year"])
display(df_immigration_exit.head())
df_immigration_exit.to_csv("data_immigration_exit.csv", sep=',', encoding='utf-8',index = False)

,Year,country_origin,name_origin,Value
804,1994,AD,Andorra,2.0
959,1995,AD,Andorra,2.0
1119,1996,AD,Andorra,3.0
1284,1997,AD,Andorra,1.0
1451,1998,AD,Andorra,2.0


In [5]:
df_immigration_entry = df_merge.drop("country_origin",1)
df_immigration_entry = df_immigration_entry.groupby(["Year","country_asylum","name_asylum"]).sum().reset_index()
df_immigration_entry= df_immigration_entry.sort_values(["country_asylum","Year"])
df_immigration_entry = df_immigration_entry[["country_asylum","Year","Value","name_asylum"]]
display(df_immigration_entry.head())
df_immigration_entry.to_csv("data_immigration_entry.csv", sep=',', encoding='utf-8',index = False)

,country_asylum,Year,Value,name_asylum
0,AE,1985,5370.0,United Arab Emirates
524,AE,1993,389.0,United Arab Emirates
647,AE,1994,448.0,United Arab Emirates
770,AE,1995,375.0,United Arab Emirates
896,AE,1996,427.0,United Arab Emirates


In [8]:
df_immigration_delta = df_immigration_entry.merge(df_immigration_exit,right_on=['Year','country_origin'], left_on=['Year','country_asylum'], how='outer')
df_immigration_delta = df_immigration_delta.fillna(0)
df_immigration_delta['value'] = df_immigration_delta.Value_x - df_immigration_delta.Value_y
df_immigration_delta = df_immigration_delta[["Year","country_asylum","value"]]
df_immigration_delta = df_immigration_delta.rename(columns={"country_asylum":"country"})

In [10]:
display(df_immigration_delta.head())

df_immigration_delta.to_csv("data_immigration_delta.csv", sep=',', encoding='utf-8',index = False)

,Year,country,value
0,1985,AE,5370.0
1,1993,AE,389.0
2,1994,AE,448.0
3,1995,AE,375.0
4,1996,AE,427.0
